# Equation of state (EOS) WorkGraph

To run this tutorial, you need to install aiida-workgraph and set up a AiiDA profile.

## Create the calcfunction task


In [7]:
from aiida import orm
from aiida_workgraph import task

# explicitly define the output socket name to match the return value of the function
@task.calcfunction(outputs=[{"name": "structures"}])
def scale_structure(structure, scales):
    """Scale the structure by the given scales."""
    atoms = structure.get_ase()
    structures = {}
    for i in range(len(scales)):
        atoms1 = atoms.copy()
        atoms1.set_cell(atoms.cell * scales[i], scale_atoms=True)
        structure = orm.StructureData(ase=atoms1)
        structures[f"s_{i}"] = structure
    return {"structures": structures}

@task.calcfunction()
# because this is a calcfunction, and the input datas are dynamic, we need use **datas.
def eos(**datas):
    """Fit the EOS of the data."""
    from ase.eos import EquationOfState

    volumes = []
    energies = []
    for _, data in datas.items():
        volumes.append(data.dict.volume)
        energies.append(data.dict.energy)
        unit = data.dict.energy_units
    #
    eos = EquationOfState(volumes, energies)
    v0, e0, B = eos.fit()
    eos = orm.Dict({"unit": unit, "v0": v0, "e0": e0, "B": B})
    return eos

## Build the workgraph
Three steps:

- create an empty WorkGraph
- add tasks: scale_structure, scf and eos.
- link the output and input sockets for the tasks.


In [8]:
from aiida_workgraph import WorkGraph, map_
from aiida_quantumespresso.calculations.pw import PwCalculation

def eos_workgraph(structure: orm.StructureData = None,
                  scales: list = None,
                  scf_inputs: dict = None):
    wg = WorkGraph("eos")
    wg.add_task(scale_structure, name="scale",
                                   structure=structure, scales=scales)
    map_(wg.tasks.scale.outputs.structures)(
        wg.add_task(PwCalculation, name=f"scf", structure="{{map_input}}"),
        wg.tasks.scf.set(scf_inputs),
    )
    wg.add_task(eos, name="eos1", datas=wg.tasks.scf.outputs.output_parameters)
    return wg

## Prepare inputs and run

### Visualize the workgraph
If you are running in a jupyter notebook, you can visualize the it directly.

In [10]:
from aiida import load_profile
from aiida.orm import Dict, load_code, load_group
from ase.build import bulk

load_profile()

si = orm.StructureData(ase=bulk("Si"))
code = load_code("qe-7.2-pw@localhost")
pw_paras = Dict({
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    })
# Load the pseudopotential family.
pseudo_family = load_group('SSSP/1.3/PBEsol/efficiency')
pseudos = pseudo_family.get_pseudos(structure=si)
#
metadata = {'options': {
                'resources': {
                'num_machines': 1,
                'num_mpiprocs_per_machine': 1,
            },
        }
    }

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
pseudos = pseudo_family.get_pseudos(structure=si)
scf_inputs = {"code": code,
        "parameters": pw_paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        }
#-------------------------------------------------------
# set the input parameters for each task
wg = eos_workgraph(si, [0.95, 1.0, 1.05], scf_inputs)
print("Waiting for the workgraph to finish...")
# wg.submit(wait=True, timeout=300)
# one can also run the workgraph directly
wg.run()
wg.to_html()
# visualize the workgraph in jupyter-notebook
# wg


Waiting for the workgraph to finish...


03/08/2025 03:34:08 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:34:08 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: tasks ready to run: scale
03/08/2025 03:34:08 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|run_tasks]: Run task: scale, type: CALCFUNCTION


------------------------------------------------------------
kwargs:  {'scales': [0.95, 1.0, 1.05], 'structure': <StructureData: uuid: 218c37ab-2600-4fc4-85d2-180490e6b5fb (pk: 41861)>}


03/08/2025 03:34:09 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_task_state]: Task: scale finished.
03/08/2025 03:34:10 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:34:10 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: tasks ready to run: map_1
03/08/2025 03:34:10 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|run_tasks]: Run task: map_1, type: MAP


------------------------------------------------------------
kwargs:  {'placeholder': 'map_input', 'source': AttributeDict({'s_0': <StructureData: uuid: b9bdc45d-f9fe-4de6-81ff-7a925906403f (pk: 41867)>, 's_1': <StructureData: uuid: 6d2fd41d-cb78-4031-8422-3ffba7b945b6 (pk: 41868)>, 's_2': <StructureData: uuid: 7b1349fb-5f03-4b2f-8564-ec49043a3cbb (pk: 41869)>})}


...

------------------------------------------------------------
kwargs:  {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 390, uuid: a0f700b9-418b-4eb4-ae0a-23dab2453ec3>, 'kpoints': <KpointsData: uuid: 885966b4-bb18-4f16-8dc7-b0fd73aeedbd (pk: 41863)>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 050fe49e-8197-47da-be0e-419d7370ab5b (pk: 41862)>, 'pseudos': {'Si': <UpfData: uuid: 0c29f605-15c0-4dd1-a8f7-9331cbf03644 (pk: 558)>}, 'structure': <StructureData: uuid: 7b1349fb-5f03-4b2f-8564-ec49043a3cbb (pk: 41869)>}


03/08/2025 03:34:18 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 41870, 41876, 41882
03/08/2025 03:34:19 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_task_state]: Task: s_0_scf finished.
03/08/2025 03:34:19 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_task_state]: Task: s_1_scf finished.
03/08/2025 03:34:20 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_task_state]: Task: s_2_scf finished.
03/08/2025 03:34:21 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_template_task_state]: Task: scf finished.
03/08/2025 03:34:21 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_zone_

------------------------------------------------------------
kwargs:  {}


03/08/2025 03:34:23 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|update_task_state]: Task: eos1 finished.
03/08/2025 03:34:23 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:34:23 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
03/08/2025 03:34:23 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41864|WorkGraphEngine|finalize]: Finalize workgraph.


{'execution_count': <Int: uuid: 8c37dec9-488f-4533-a11b-3c08fcba8e3a (pk: 41890) value: 1>}

Print out the results:

In [11]:
data = wg.tasks["eos1"].outputs["result"].value.get_dict()
print('B: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))

B: 0.53596259211164
v0: 41.134100879971
e0: -308.19240692174
v0: 41.134100879971


### Use it inside another workgraph
In the above example, we call the `eos_workflow` directly to generate the workgraph and submit it. We can also use it as a task inside another workgraph. In this way, we can create a nested workflow.

For example, we want to combine relax with eos.

We can use the `graph_builder` decorator. The Graph Builder allow user to create a dynamic workflow based on the input value, as well as nested workflows.

In [12]:
from aiida_workgraph import WorkGraph, task

@task.graph_builder(outputs=[{"name": "result", "from": "eos.result"}])
def eos_workgraph(structure: orm.StructureData = None,
                  scales: list = None,
                  scf_inputs: dict = None):
    wg = WorkGraph("eos")
    wg.add_task(scale_structure, name="scale",
                                   structure=structure, scales=scales)
    map_(wg.tasks.scale.outputs.structures)(
        wg.add_task(PwCalculation, name=f"scf", structure="{{map_input}}"),
        wg.tasks.scf.set(scf_inputs),
    )
    wg.add_task(eos, name="eos", datas=wg.tasks.scf.outputs.output_parameters)
    return wg

In [13]:
from aiida_workgraph import WorkGraph
from copy import deepcopy
from aiida_quantumespresso.calculations.pw import PwCalculation

#-------------------------------------------------------
relax_pw_paras = deepcopy(pw_paras)
relax_pw_paras["CONTROL"]["calculation"] = "vc-relax"
relax_inputs = {
        "structure": si,
        "code": code,
        "parameters": relax_pw_paras,
        "kpoints": kpoints,
        "pseudos": pseudos,
        "metadata": metadata
        }
#-------------------------------------------------------
wg = WorkGraph("relax_eos")
wg.add_task(PwCalculation, name="relax")
wg.tasks.relax.set(relax_inputs)
eos_wg_task = wg.add_task(eos_workgraph, name="eos1",
                          structure=wg.tasks.relax.outputs.output_structure,
                          scales=[0.95, 1.0, 1.05],
                          scf_inputs=scf_inputs)
#-------------------------------------------------------
print("Waiting for the workgraph to finish...")
wg.run()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wg.tasks["eos1"].outputs["result"].value.get_dict()))
wg.to_html()
# visualize the workgraph in jupyter-notebook
# wg

Waiting for the workgraph to finish...


03/08/2025 03:38:41 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:38:41 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|continue_workgraph]: tasks ready to run: relax
03/08/2025 03:38:41 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|run_tasks]: Run task: relax, type: CALCJOB


------------------------------------------------------------
kwargs:  {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 390, uuid: a0f700b9-418b-4eb4-ae0a-23dab2453ec3>, 'kpoints': <KpointsData: uuid: 885966b4-bb18-4f16-8dc7-b0fd73aeedbd (pk: 41863)>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 212b207d-51fa-4cd5-9a3c-60a034a9ece6 (pk: 41891)>, 'pseudos': {'Si': <UpfData: uuid: 0c29f605-15c0-4dd1-a8f7-9331cbf03644 (pk: 558)>}, 'structure': <StructureData: uuid: 218c37ab-2600-4fc4-85d2-180490e6b5fb (pk: 41861)>}


03/08/2025 03:38:43 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 41893
03/08/2025 03:39:20 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|update_task_state]: Task: relax finished.
03/08/2025 03:39:21 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:39:21 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|continue_workgraph]: tasks ready to run: eos1
03/08/2025 03:39:21 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|run_tasks]: Run task: eos1, type: graph_builder


------------------------------------------------------------
kwargs:  {'scales': [0.95, 1.0, 1.05], 'scf_inputs': {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 390, uuid: a0f700b9-418b-4eb4-ae0a-23dab2453ec3>, 'kpoints': <KpointsData: uuid: 885966b4-bb18-4f16-8dc7-b0fd73aeedbd (pk: 41863)>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 050fe49e-8197-47da-be0e-419d7370ab5b (pk: 41862)>, 'pseudos': {'Si': <UpfData: uuid: 0c29f605-15c0-4dd1-a8f7-9331cbf03644 (pk: 558)>}}, 'structure': <StructureData: uuid: c73a2ca7-2545-4e4a-8414-3f3e677ae0f4 (pk: 41898)>}


...

------------------------------------------------------------
kwargs:  {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 390, uuid: a0f700b9-418b-4eb4-ae0a-23dab2453ec3>, 'kpoints': <KpointsData: uuid: 885966b4-bb18-4f16-8dc7-b0fd73aeedbd (pk: 41863)>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 050fe49e-8197-47da-be0e-419d7370ab5b (pk: 41862)>, 'pseudos': {'Si': <UpfData: uuid: 0c29f605-15c0-4dd1-a8f7-9331cbf03644 (pk: 558)>}, 'structure': <StructureData: uuid: b4435465-85e7-4b63-b9be-2d5ba3f66e94 (pk: 41905)>}


03/08/2025 03:39:31 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 41906, 41907, 41908
03/08/2025 03:39:45 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|update_task_state]: Task: s_0_scf finished.
03/08/2025 03:39:46 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:39:46 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
03/08/2025 03:39:46 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 41907, 41908
03/08/2025 03:39:51 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngin

------------------------------------------------------------
kwargs:  {}


/home/xing/apps/miniforge3/envs/aiida/lib/python3.11/site-packages/ase/eos.py:307: RankWarning: Polyfit may be poorly conditioned
  fit0 = np.poly1d(np.polyfit(self.v**-(1 / 3), self.e, 3))
03/08/2025 03:39:55 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|update_task_state]: Task: eos finished.
03/08/2025 03:39:55 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/08/2025 03:39:55 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
03/08/2025 03:39:55 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41900|WorkGraphEngine|finalize]: Finalize workgraph.
03/08/2025 03:39:56 PM <241352> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [41892|WorkGraphEngine|update_task_state]: Task: eos1 finished.
03/08


Result: 
B: 0.51887865638682
v0: 41.16774289795
e0: -308.19005654624
v0: 41.16774289795


## Summary
There are many ways to create the workflow using graph builder. For example, one can add the relax step inside the `eos_workgraph`, and add a `run_relax` argument to control the logic.